In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['RD']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('radial diffusivity', 224),
 ('retinal detachment', 133),
 ('rhabdomyosarcoma', 106),
 ('regulatory domain', 72),
 ('reading disability', 58),
 ('regular diet', 50),
 ('retinal degeneration', 44),
 ('renal dysfunction', 41),
 ('renal denervation', 40),
 ('risk difference', 28),
 ('repressor domain', 27),
 ('receiver domain', 22),
 ('radial', 21),
 ('retinal determination', 19),
 ('reward dependence', 19),
 ('runt domain', 17),
 ('restrictive dermopathy', 16),
 ('respiratory disease', 16),
 ('reading difficulties', 14),
 ('repression domain', 13),
 ('reading disorder', 12),
 ('radicicol', 11),
 ('rhabdomyosarcoma cell', 10),
 ('rectal distension', 9),
 ('reaction diffusivity', 9),
 ('rhabdomyosarcoma cell line', 9),
 ('respiratory depression', 8),
 ('recession depth', 8),
 ('residual disease', 8),
 ('radiation damping', 7),
 ('relaxation dispersion', 7),
 ('root diameter', 7),
 ('respiratory distress', 7),
 ('rebound depolarization', 7),
 ('riboflavin deficient', 7),
 ('repeat domain'

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-09-25 14:47:31] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'benzaldehyde lyase': 'MESH:C059416',
  'betaine aldehyde': 'CHEBI:CHEBI:15710',
  'bile salt activity lipase': 'HGNC:1848',
  'bioartificial liver': 'MESH:D019164',
  'blood alcohol levels': 'ungrounded',
  'breath alcohol levels': 'ungrounded',
  'british anti lewisite': 'CHEBI:CHEBI:64198',
  'brochoalveolar lavage': 'MESH:D018893',
  'bronchalveolar lavage': 'MESH:D018893',
  'bronchial alveolar lavage': 'MESH:D018893',
  'bronchial lavage': 'MESH:D018893',
  'bronchio alveolar lavage': 'MESH:D018893',
  'bronchiolar lavage': 'MESH:D018893',
  'broncho alveolar lavage': 'MESH:D018893',
  'bronchoalveolar': 'MESH:D018893',
  'bronchoalveolar fluid': 'MESH:D018893',
  'bronchoalveolar larvage': 'MESH:D018893',
  'bronchoalveolar lavage': 'MESH:D018893'},
 {'MESH:C059416': 'benzaldehyde lyase',
  'CHEBI:CHEBI:15710': 'betaine aldehyde',
  'HGNC:1848': 'CEL',
  'MESH:D019164': 'Liver, Artificial',
  'CHEBI:CHEBI:64198': 'dimercaprol',
  'MESH:D018893': 'Bronchoalveolar Lavage'},
 ['H

In [6]:
grounding_map, names, pos_labels = grounding_map, names, pos_labels = [{'racial discrimination': 'MESH:D063505',
  'radial': 'ungrounded',
  'radial diffusivity': 'radial_diffusivity',
  'radiation damping': 'ungrounded',
  'radiation dermatitis': 'ungrounded',
  'radiation dose': 'ungrounded',
  'radicicol': 'CHEBI:CHEBI:556075',
  'radiology department': 'ungrounded',
  'raphe dorsalis': 'ungrounded',
  'rate difference': 'ungrounded',
  'reaction diffusivity': 'ungrounded',
  'reading': 'ungrounded',
  'reading depth': 'ungrounded',
  'reading difficulties': 'ungrounded',
  'reading disability': 'MESH:D004410',
  'reading disorder': 'MESH:D004410',
  'rebound depolarization': 'ungrounded',
  'receiver domain': 'ungrounded',
  'recession depth': 'ungrounded',
  'recombinase domain': 'ungrounded',
  'recommended dose': 'ungrounded',
  'recruitment domain': 'ungrounded',
  'rectal distension': 'ungrounded',
  'rectal distention': 'ungrounded',
  'reduction dechlorination': 'ungrounded',
  'regimen distress': 'ungrounded',
  'registered dietitian': 'ungrounded',
  'regression discontinuity': 'ungrounded',
  'regular chow diet': 'MESH:D004032',
  'regular diet': 'ungrounded',
  'regulatory domain': 'ungrounded',
  'related demethylation': 'ungrounded',
  'related deprivation': 'ungrounded',
  'related disease': 'ungrounded',
  'relaxation dispersion': 'ungrounded',
  'rem sleep deprivation': 'MESH:D012892',
  'renal denervation': 'ungrounded',
  'renal dysfunction': 'MESH:D051437',
  'renal sympathetic denervation': 'MESH:D013562',
  'repeat domain': 'ungrounded',
  'replant disease': 'ungrounded',
  'replication defective': 'ungrounded',
  'repression domain': 'ungrounded',
  'repressor domain': 'ungrounded',
  'residual disease': 'ungrounded',
  'resistant disease': 'ungrounded',
  'respiration deficient': 'ungrounded',
  'respiratory deficient': 'ungrounded',
  'respiratory depression': 'EFO:0009842',
  'respiratory disease': 'MESH:D012140',
  'respiratory distress': 'ungrounded',
  'restrictive dermopathy': 'ungrounded',
  'restrictive diffusivity': 'ungrounded',
  'resveratrol dimer': 'ungrounded',
  'reticular dysgenesis': 'MESH:C538361',
  'retinal degeneration': 'MESH:D012162',
  'retinal detachment': 'MESH:D012163',
  'retinal determination': 'ungrounded',
  'retinal dystrophy': 'MESH:D058499',
  'retinoid deficient': 'ungrounded',
  'reward dependence': 'ungrounded',
  'reward drive': 'ungrounded',
  'rhabdomyosarcoma': 'MESH:D012208',
  'rhabdomyosarcoma cell': 'ungrounded',
  'rhabdomyosarcoma cell line': 'ungrounded',
  'rheumatic disease': 'ungrounded',
  'rhizoma drynariae': 'ungrounded',
  'rhododendrol': 'MESH:C115945',
  'riboflavin deficient': 'ungrounded',
  'riccardin d': 'MESH:C547185',
  'rice duck farming': 'ungrounded',
  'risk difference': 'ungrounded',
  'root diameter': 'ungrounded',
  'rosa damascena l': 'ungrounded',
  'roughing disorder': 'ungrounded',
  'runt domain': 'ungrounded'},
 {'MESH:D063505': 'Racism',
  'radial_diffusivity': 'radial_diffusivity',
  'CHEBI:CHEBI:556075': 'radicicol',
  'MESH:D004410': 'Dyslexia',
  'MESH:D004032': 'Diet',
  'MESH:D012892': 'Sleep Deprivation',
  'MESH:D051437': 'Renal Insufficiency',
  'MESH:D013562': 'Sympathectomy',
  'EFO:0009842': 'respiratory depression',
  'MESH:D012140': 'Respiratory Tract Diseases',
  'MESH:C538361': 'Reticular dysgenesis',
  'MESH:D012162': 'Retinal Degeneration',
  'MESH:D012163': 'Retinal Detachment',
  'MESH:D058499': 'Retinal Dystrophies',
  'MESH:D012208': 'Rhabdomyosarcoma',
  'MESH:C115945': 'rhododendrol',
  'MESH:C547185': 'riccardin D'},
 ['CHEBI:CHEBI:556075',
  'MESH:D012162',
  'MESH:D012163',
  'MESH:D012208',
  'MESH:D051437']]

In [7]:
excluded_longforms = []

In [8]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [9]:
additional_entities = {'HGNC:13974': ['NELFE', ['RD', 'RDBD', 'RDB']]}

In [10]:
unambiguous_agent_texts = {}

In [11]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [12]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [13]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [14]:
intersection1

[('HGNC:13974', 'HGNC:13974', 74)]

In [15]:
intersection2

[('ungrounded', 'HGNC:13974', 0),
 ('MESH:D004410', 'HGNC:13974', 0),
 ('MESH:D051437', 'HGNC:13974', 0),
 ('MESH:D012208', 'HGNC:13974', 0),
 ('MESH:D012163', 'HGNC:13974', 0),
 ('CHEBI:CHEBI:556075', 'HGNC:13974', 0),
 ('MESH:D012162', 'HGNC:13974', 0),
 ('radial_diffusivity', 'HGNC:13974', 0),
 ('MESH:D012140', 'HGNC:13974', 0),
 ('MESH:D058499', 'HGNC:13974', 0),
 ('MESH:C538361', 'HGNC:13974', 0),
 ('MESH:D013562', 'HGNC:13974', 0),
 ('MESH:C547185', 'HGNC:13974', 0),
 ('MESH:D012892', 'HGNC:13974', 0),
 ('EFO:0009842', 'HGNC:13974', 0),
 ('MESH:C115945', 'HGNC:13974', 0),
 ('MESH:D004032', 'HGNC:13974', 0),
 ('MESH:D063505', 'HGNC:13974', 0)]

In [16]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [17]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [18]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-29 02:18:14] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-29 02:18:40] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.8168638711735381 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [19]:
classifier.stats

{'label_distribution': {'ungrounded': 399,
  'MESH:D004410': 41,
  'MESH:D051437': 24,
  'MESH:D012208': 75,
  'MESH:D012163': 87,
  'CHEBI:CHEBI:556075': 7,
  'MESH:D012162': 29,
  'radial_diffusivity': 138,
  'MESH:D012140': 7,
  'MESH:D058499': 3,
  'MESH:C538361': 2,
  'MESH:D013562': 5,
  'MESH:C547185': 3,
  'MESH:D012892': 1,
  'EFO:0009842': 5,
  'MESH:C115945': 5,
  'MESH:D004032': 1,
  'MESH:D063505': 2,
  'HGNC:13974': 10},
 'f1': {'mean': 0.816864, 'std': 0.08313},
 'precision': {'mean': 0.870062, 'std': 0.07813},
 'recall': {'mean': 0.789177, 'std': 0.10313},
 'ungrounded': {'f1': {'mean': 0.865408, 'std': 0.017948},
  'pr': {'mean': 0.909652, 'std': 0.03156},
  'rc': {'mean': 0.826733, 'std': 0.032352}},
 'MESH:D013562': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'MESH:D058499': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'MESH:D00441

In [20]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [21]:
disamb.dump(model_name, results_path)

In [22]:
print(disamb.info())

Disambiguation model for RD

Produces the disambiguations:
	Diet	MESH:D004032
	Dyslexia	MESH:D004410
	NELFE*	HGNC:13974
	Racism	MESH:D063505
	Renal Insufficiency*	MESH:D051437
	Respiratory Tract Diseases	MESH:D012140
	Reticular dysgenesis	MESH:C538361
	Retinal Degeneration*	MESH:D012162
	Retinal Detachment*	MESH:D012163
	Retinal Dystrophies	MESH:D058499
	Rhabdomyosarcoma*	MESH:D012208
	Sleep Deprivation	MESH:D012892
	Sympathectomy	MESH:D013562
	radial_diffusivity	radial_diffusivity
	radicicol*	CHEBI:CHEBI:556075
	respiratory depression	EFO:0009842
	rhododendrol	MESH:C115945
	riccardin D	MESH:C547185

Class level metrics:
--------------------
Grounding                 	Count	F1     
                Ungrounded	399	0.86541
        radial_diffusivity	138	0.90139
        Retinal Detachment*	 87	0.94745
          Rhabdomyosarcoma*	 75	0.77586
                  Dyslexia	 41	0.89516
      Retinal Degeneration*	 29	0.62848
       Renal Insufficiency*	 24	0.80318
                     NELFE*	 10	

In [23]:
model_to_s3(disamb)